In [1]:
import pandas as pd
import numpy as np
import os
import boto3

from config import ACCESS_KEY, SECRET_KEY, TOKEN
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('display.max_colwidth',None)


Folder = '/home/boris/Documents/Work/metrics_warranty'

In [2]:
data = pd.read_csv(f'{Folder}/1.csv', sep=";") #выгрузка с ID в первом столбце

list_of_columns=[
    'Дата начала',
    'Дата отправки',
    'Дата завершения',
    'Заявка принята на склад',
    'Дата начала гарантийного периода',
    'Новая заявка',
    'Начало ремонта',
    'Отложено',
    'Выходной контроль',
    'Готово к отправке'
]

for i in list_of_columns:
    data[i]=pd.to_datetime(data[i],dayfirst=True)
    
data_products = pd.read_csv(f'{Folder}/2.csv', sep=";") #выгрузка с товаром в первом столбце
data_products = data_products[['Товар', 'ID']]
data_products = data_products.fillna('')

data_products_grouped = data_products.groupby("ID")["Товар"].apply(list).reset_index()
data_merged = data.merge(
    data_products_grouped,
    on='ID',
    how='left'
)

list_of_works = ['Диагностика, выходной контроль',
    'Дополнительные работы 20 мин.',
    'Компонентный ремонт',
    'Микропроцессорный ремонт',
    'Замена корпуса Один (пластик)',
    'Замена корпуса Один С (металл)',
    'Замена задней крышки',
    'Замена объектива модуля камеры',
    'Замена модуля камеры в сборе',
    'Прошивка камеры',
    'Прошивка домофона',
    'Присвоение ID домофона',
    'Замена микрофона',
    'Замена RFID-антенны (NFC, Em-marine)',
    'Замена мембраны',
    'Замена динамика',
    'Замена стекла',
    'Замена материнской платы',
    'Замена транзистора',
    'Замена диода', 
    'Замена свитча',
    'Замена визнета',
    'Замена кодека',
    'Замена процессора',
    'Замена модуля BLE',
    'Восстановление токопроводящих дорожек',
    'Замена Ик-диода',
    'Замена датчика освещенности.',
    'Восстановление LAN порта',
    'Ремонт модуля камеры'
    
]

for columns in list_of_works:
    data_merged[columns] = 0

/tmp/ipykernel_1107/2711461572.py:1: DtypeWarning: Columns (17,20,22,24,27,38,39,40,41,42,53) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f'{Folder}/1.csv', sep=";") #выгрузка с ID в первом столбце
/tmp/ipykernel_1107/2711461572.py:19: DtypeWarning: Columns (10,13,14,17,18,20,23,25,27,30,41,42,43,44,45,56,58,65) have mixed types. Specify dtype option on import or set low_memory=False.
  data_products = pd.read_csv(f'{Folder}/2.csv', sep=";") #выгрузка с товаром в первом столбце


In [3]:
data_products_grouped.to_excel(f'{Folder}/remade/data_products_grouped.xlsx',index=False)

In [4]:
def products_change(x):
    for i in range(len(x)):
        if x[i].strip() in ['Диагностика, выходной контроль/Сброс настроек']:
            x[i] = 'Диагностика, выходной контроль'
        
        if x[i].strip() in ['Дополнительные работы 30 мин.']:
            x[i] = 'Дополнительные работы 20 мин.'

        if x[i].strip() in ['Корпус Один']:
            x[i] = 'Замена корпуса Один (пластик)'

        if x[i].strip() in ['Замена корпуса','Ремкомплект ОДИН-С','Корпус','Корпус  ОДИН-С',]:
            x[i] = 'Замена корпуса Один С (металл)'

        if x[i].strip() in ['Камера','Замена камеры']:
            x[i] = 'Замена модуля камеры в сборе'

        if x[i].strip() in ['Восстановление программного обеспечения']:
            x[i] = 'Прошивка домофона'

        if x[i].strip() in ['Микрофон Один','Микрофон Один-С']:
            x[i] = 'Замена микрофона'
        
        if x[i].strip() in ['Мембрана силиконовая для кнопок']:
            x[i] = 'Замена мембраны'

        if x[i].strip() in ['Динамик Один','Динамик Один-С','Динамик Один-С']:
            x[i] = 'Замена динамика'

        if x[i].strip() in ['Стекло Один','Стекло  Один С']:
            x[i] = 'Замена стекла'

        if x[i].strip() in ['Диод защитный','Защитный диод SMAJ24A-TR']:
            x[i] = 'Замена диода'

        if x[i].strip() in ['Замена свитча Один','Замена свитча Ио','Замена свитча Ио']:
            x[i] = 'Замена свитча'

        x[i] = x[i].strip()
        if len(x[i])>0:
            x[i][0].upper() + x[i][1:]  
    return x

data_merged['Товар'] = data_merged['Товар'].apply(lambda x: products_change(x))


In [5]:
cell_number = 0
for cell in data_merged['Товар']:
    for works in cell:
        if works in list_of_works:
            data_merged.loc[cell_number, works] = 1
    cell_number += 1

for colluns in list_of_works:
    data_merged[colluns] = data_merged[colluns].astype('int64')

data_merged['ID'] = data_merged['ID'].astype('str')

data_smart_processes = pd.read_csv(f'{Folder}/3.csv', sep=";") #выгрузка смарт процессоы

data_smart_processes = data_smart_processes[['ID сделки','Выявленная неисправность']]
data_smart_processes['ID сделки'] = data_smart_processes['ID сделки'].astype('str')
data_smart_processes['ID сделки'] = data_smart_processes['ID сделки'].apply(lambda x: x[0:-2])
data_smart_processes_grouped = data_smart_processes.groupby('ID сделки')['Выявленная неисправность'].apply(list).reset_index()
data_smart_processes_grouped.loc[data_smart_processes_grouped[data_smart_processes_grouped['ID сделки'] == 'n'].index[0],['ID сделки']] = "0"
data_smart_processes_grouped.rename(columns={'ID сделки':'ID'}, inplace=True)

In [6]:
list_of_defects= ['nan',
'НД → ЗВУК: В домофоне нет звука голоса, есть только системные звуки',
'НД → ЗВУК: Домофон не издает никаких звуков',
'НД → ЗВУК: Голос или звук искажен (фон, треск, свист, шум)',
'НД → ЗВУК: Голос или звук тихий даже после настройки',
'НД → ЗВУК: Нет голоса в абонентской трубке или в приложении',
'НД → ЗАМОК (ЭМЗ): Замок не отпускает или отпускает частично',
'НД → ЗАМОК (ЭМЗ): Замок не держит',
'НД → ЗАМОК (ЭМЗ): Ключ периодически или постоянно не срабатывает',
'НД → СЕТЬ, ИНТЕРНЕТ: В сеть выходит или домофон, или камера, но не вместе',
'НД → СЕТЬ, ИНТЕРНЕТ: Ни камера, ни домофон не выходит в сеть (мигает крестик)',
'НД → СЕТЬ, ИНТЕРНЕТ: Зависает при подключении интернета',
'НД → ВЫЗОВЫ, КОММУТАТОР: Нет вызова ни в какую квартиру',
'НД → ВЫЗОВЫ, КОММУТАТОР: Нет реакции на снятие абонентской трубки',
'НД → ВЫЗОВЫ, КОММУТАТОР: Не срабатывает нажатие кнопки или кнопок (цифры, вызов, сброс)',
'НД → НЕРАБОТОСПОСОБНА: Не включается после подачи питания',
'НД → НЕРАБОТОСПОСОБНА: Постоянная перезагрузка при исправном блоке питания',
'НД → НЕРАБОТОСПОСОБНА: Периодически зависает или перезагружается',
'НД → НЕРАБОТОСПОСОБНА: Коррозия платы',
'НД → НЕРАБОТОСПОСОБНА: Воздействие перенапряжения',
'НД → НЕРАБОТОСПОСОБНА: Вандализм',
'НД → КАМЕРА: Нет видеопотока с камеры (в веб-сервисах и в приложении)',
'НД → КАМЕРА: Плохое качество видео (изменение цвета, размытие, другое)',
'НД → КАМЕРА: Камера не работает, в сеть не выходит',
'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Нет видеопотока с камеры (в веб-сервисах и в приложении)',
'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Плохое качество видео (изменение цвета, размытие, другое)',
'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не выходить в сеть',
'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не работает, в сеть не выходит',
'НДО → БЛОК ПИТАНИЯ: Не включается',
'НДО → БЛОК ПИТАНИЯ: Не обеспечивает нужные параметры',
'НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не включается',
'НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не обеспечивает нужные параметры',
'ДРУГОЕ: Заявка на замену корпуса',
'ДРУГОЕ: Неисправности не выявлены']

# data_smart_processes_grouped.loc[data_smart_processes_grouped[data_smart_processes_grouped['ID'] == 'n'].index[0],['ID']] = "0"
data_merged['ID'] = data_merged['ID'].astype(int)
data_smart_processes_grouped['ID'] = data_smart_processes_grouped['ID'].astype(int)

for columns in list_of_defects:
    data_smart_processes_grouped[columns] = ''

line_count = 0
for lines in data_smart_processes_grouped['Выявленная неисправность']:
    for defect in lines:
        data_smart_processes_grouped.loc[line_count,[defect]] = '1'
    line_count += 1

data_smart_processes_grouped = data_smart_processes_grouped.drop(['Выявленная неисправность','nan'], axis=1)

In [7]:
# data_smart_processes_grouped.to_excel(f'{Folder}/remade/data_smart_processes_grouped.xlsx',index=False)

In [8]:
data_merged = data_merged.merge(
    data_smart_processes_grouped,
    on='ID',
    how='left'
)

In [9]:
# data_merged.to_excel(f'{Folder}/remade/data_merged.xlsx',index=False)


In [10]:
data_merged['Дополнительные дефекты'] = data_merged['Дополнительные дефекты'].fillna('')

list_of_additional_deffect = [
    'Гроза',
    'Коррозия',
    'BLE',
    'Ключи (не подтвердилось)',
    'КЗ ИК'
]

for i in range(data_merged.shape[0]):
    for j in data_merged.loc[i,'Дополнительные дефекты'].split(','):
        if j.strip() in list_of_additional_deffect:
            data_merged.loc[i,[f'Выявлено: {j.strip()}']]= '1'

In [11]:
list_of_defects3 = [
    'Выявлено: Звук',
    'Выявлено: Замок (ЭМЗ)',
    'Выявлено: Сеть, Интернет',
    'Выявлено: Вызовы, коммутатор',
    'Выявлено: Неработоспособна',
    'Выявлено: Камера',
    'Выявлено: Неисправность Домофона: Другое',
    'Выявлено: Камера видеонаблюдения',
    'Выявлено: Блок питания',
    'Выявлено: Бесконтактная кнопка выхода',
    'Выявлено: Неполадки другого оборудования: Другое']

data_merged[list_of_defects3] = data_merged[list_of_defects3].fillna('')
rows_count = data_merged.shape[0]
    

for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → ЗВУК: В домофоне нет звука голоса, есть только системные звуки'] == '1' or\
        data_merged.loc[cell_number,'НД → ЗВУК: Домофон не издает никаких звуков'] == '1' or\
        data_merged.loc[cell_number,'НД → ЗВУК: Голос или звук искажен (фон, треск, свист, шум)'] == '1' or\
        data_merged.loc[cell_number,'НД → ЗВУК: Голос или звук тихий даже после настройки'] == '1' or\
        data_merged.loc[cell_number,'НД → ЗВУК: Нет голоса в абонентской трубке или в приложении'] == '1':
        data_merged.loc[cell_number,'Выявлено: Звук'] = '1'
       

for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → СЕТЬ, ИНТЕРНЕТ: В сеть выходит или домофон, или камера, но не вместе'] == '1':
        data_merged.loc[cell_number,'Выявлено: Сеть, Интернет'] = '1'
    elif data_merged.loc[cell_number,'НД → СЕТЬ, ИНТЕРНЕТ: Ни камера, ни домофон не выходит в сеть (мигает крестик)'] == '1':
        data_merged.loc[cell_number,'Выявлено: Сеть, Интернет'] = '1'
    elif data_merged.loc[cell_number,'НД → СЕТЬ, ИНТЕРНЕТ: Зависает при подключении интернета'] == '1':
        data_merged.loc[cell_number,'Выявлено: Сеть, Интернет'] = '1'

    
for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → ЗАМОК (ЭМЗ): Замок не отпускает или отпускает частично'] == '1' or\
    data_merged.loc[cell_number,'НД → ЗАМОК (ЭМЗ): Замок не держит'] == '1' or\
    data_merged.loc[cell_number,'НД → ЗАМОК (ЭМЗ): Ключ периодически или постоянно не срабатывает'] == '1':
        data_merged.loc[cell_number,'Выявлено: Замок (ЭМЗ)'] = '1'


for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → ВЫЗОВЫ, КОММУТАТОР: Нет вызова ни в какую квартиру'] == '1' or\
    data_merged.loc[cell_number,'НД → ВЫЗОВЫ, КОММУТАТОР: Нет реакции на снятие абонентской трубки'] == '1':
        data_merged.loc[cell_number,'Выявлено: Вызовы, коммутатор'] = '1'

for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → ВЫЗОВЫ, КОММУТАТОР: Не срабатывает нажатие кнопки или кнопок (цифры, вызов, сброс)'] == '1':
        data_merged.loc[cell_number,'Выявлено: Номеронабиратель'] = '1'
       
for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → НЕРАБОТОСПОСОБНА: Не включается после подачи питания'] == '1' or\
    data_merged.loc[cell_number,'НД → НЕРАБОТОСПОСОБНА: Постоянная перезагрузка при исправном блоке питания'] == '1' or\
    data_merged.loc[cell_number,'НД → НЕРАБОТОСПОСОБНА: Периодически зависает или перезагружается'] == '1' or\
    data_merged.loc[cell_number,'НД → НЕРАБОТОСПОСОБНА: Коррозия платы'] == '1' or\
    data_merged.loc[cell_number,'НД → НЕРАБОТОСПОСОБНА: Воздействие перенапряжения'] == '1' or\
    data_merged.loc[cell_number,'НД → НЕРАБОТОСПОСОБНА: Вандализм'] == '1':
        data_merged.loc[cell_number,'Выявлено: Неработоспособна'] = '1'
       

for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → КАМЕРА: Нет видеопотока с камеры (в веб-сервисах и в приложении)'] == '1' or\
    data_merged.loc[cell_number,'НД → КАМЕРА: Плохое качество видео (изменение цвета, размытие, другое)'] == '1' or\
    data_merged.loc[cell_number,'НД → КАМЕРА: Камера не работает, в сеть не выходит'] == '1':
        data_merged.loc[cell_number,'Выявлено: Камера'] = '1'


for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Нет видеопотока с камеры (в веб-сервисах и в приложении)'] == '1' or\
    data_merged.loc[cell_number,'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Плохое качество видео (изменение цвета, размытие, другое)'] == '1' or\
    data_merged.loc[cell_number,'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не выходить в сеть'] == '1' or\
    data_merged.loc[cell_number,'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не работает, в сеть не выходит'] == '1':
        data_merged.loc[cell_number,'Выявлено: Камера видеонаблюдения'] = '1'

        
for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НДО → БЛОК ПИТАНИЯ: Не включается'] == '1' or\
    data_merged.loc[cell_number,'НДО → БЛОК ПИТАНИЯ: Не обеспечивает нужные параметры'] == '1':
        data_merged.loc[cell_number,'Выявлено: Блок питания'] = '1'

        
for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не включается'] == '1' or\
    data_merged.loc[cell_number,'НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не обеспечивает нужные параметры'] == '1':
        data_merged.loc[cell_number,'Выявлено: Бесконтактная кнопка выхода'] = '1'

for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'ДРУГОЕ: Заявка на замену корпуса'] == '1' or\
    data_merged.loc[cell_number,'ДРУГОЕ: Заявка на замену корпуса'] == '1':
        data_merged.loc[cell_number,'Выявлено: Неполадки другого оборудования: Другое'] = '1'


In [12]:
# data_merged.to_excel(f'{Folder}/remade/metrics_warranty3.xlsx', index=False)

In [13]:
data_merged.to_csv(f'{Folder}/remade/metrics_warranty.csv', index=False, sep=',')

In [14]:
ACCESS_KEY = ACCESS_KEY
SECRET_KEY = SECRET_KEY
BUCKET_NAME = "dwh-asgard"
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow")


def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )
    
def upload_dump_to_s3():
    get_s3_instance().upload_file(
        Filename= f'{Folder}/remade/metrics_warranty.csv',
        Bucket=BUCKET_NAME,
        Key=f"metrics_dir_warranty/metrics_warranty.csv"
    )

get_s3_instance()
upload_dump_to_s3()

In [15]:
session = boto3.session.Session()

session = session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net')

In [16]:
BUCKET_NAME = "dwh-asgard"

session.upload_file(
        Filename= f'{Folder}/remade/metrics_warranty.csv',
        Bucket=BUCKET_NAME,
        Key=f"metrics_dir_warranty/metrics_warranty.csv")